In [4]:
import os
os.environ['HF_HOME'] = '/vol/tensusers3/jdusseljee/hf_home'

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
from tqdm import tqdm
import json

In [5]:
class paths:
  full_audio_dir = '/vol/tensusers3/jdusseljee/2324-asr/datasets/addresso/train/concatenated'
  transcript_dir = '/vol/tensusers3/jdusseljee/2324-asr/datasets/addresso/train/whisper_med_transcript'

In [6]:
device = "cuda:6" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "distil-whisper/distil-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)

In [9]:
result = pipe('../datasets/addresso/train/concatenated/S001.wav', generate_kwargs={"language": "english"})

In [10]:
if not os.path.exists(paths.transcript_dir):
  os.makedirs(paths.transcript_dir)

for file in tqdm(os.listdir(paths.full_audio_dir)):
  result = pipe(f'{paths.full_audio_dir}/{file}')

  with open(f"{paths.transcript_dir}/{file.split('.')[0]}.json", 'w') as f:
    json.dump(result, f)

  0%|          | 0/108 [00:00<?, ?it/s]

  7%|▋         | 8/108 [00:12<02:15,  1.36s/it]--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/vol/tensusers3/jdusseljee/2324-asr/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/vol/tensusers3/jdusseljee/2324-asr/.ven